<a href="https://colab.research.google.com/github/BradleyMerrillT/Indiana-Expungement/blob/main/EXIndianaData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

EX, the expungement app for Indiana ex-offenders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#imports
import pandas as pd
import re
import numpy as np
import math
import zipfile
import os
import pickle

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

##Import

In [ ]:
#note on line 19 I had to add a skip; will need a better solution
#https://stackoverflow.com/questions/18039057/pandas-parser-cparsererror-error-tokenizing-data
#see text note re 5 rows in cell below
def read_multiple_csv_from_zip(zip_filepath):
    """
    Reads multiple CSV files from a zip archive into a single pandas DataFrame.

    Args:
        zip_filepath (str): The path to the zip file.

    Returns:
        pandas.DataFrame: A DataFrame containing the data from all CSV files in the zip archive.
                          Returns an empty DataFrame if no CSV files are found or if an error occurs.
    """
    all_data = {}
    names = ["df1", "df2", "df3", "df4", "df5"]
    names2 = ["df1", "df2", "df3", "df4", "df5"]
    x=-1
    try:
        with zipfile.ZipFile(zip_filepath, 'r') as z:
            for filename in z.namelist():
                x=x+1
                if filename.endswith('.txt'):
                    with z.open(filename) as text_file:
                        names[x] = pd.read_csv(text_file, sep='|', on_bad_lines='warn') #, on_bad_lines='skip'
                        all_data[names2[x]] =names[x]
        if all_data:
            return all_data
        else:
            return pd.DataFrame()
    except FileNotFoundError:
         print(f"Error: The file '{zip_filepath}' was not found.")
         return pd.DataFrame()
    except zipfile.BadZipFile:
        print(f"Error: The file '{zip_filepath}' is not a valid zip file.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

In [ ]:
# usage:
zip_file_path = '/content/drive/MyDrive/Expungement/OneDrive_1_4-8-2025.zip'
final_dataframe = read_multiple_csv_from_zip(zip_file_path)


<ipython-input-4-678337d07cc7>:25: ParserWarning: Skipping line 1967950: expected 17 fields, saw 65
Skipping line 1967951: expected 17 fields, saw 116
Skipping line 1969097: expected 17 fields, saw 25
Skipping line 1969099: expected 17 fields, saw 22
Skipping line 1972652: expected 17 fields, saw 41

  names[x] = pd.read_csv(text_file, sep='|', on_bad_lines='warn') #, on_bad_lines='skip'
<ipython-input-4-678337d07cc7>:25: ParserWarning: Skipping line 2705141: expected 3 fields, saw 4

  names[x] = pd.read_csv(text_file, sep='|', on_bad_lines='warn') #, on_bad_lines='skip'


Missing 5 charges--there must be "|" s in those rows

Skipping line 1967950: expected 17 fields, saw 65
Skipping line 1967951: expected 17 fields, saw 116
Skipping line 1969097: expected 17 fields, saw 25
Skipping line 1969099: expected 17 fields, saw 22
Skipping line 1972652: expected 17 fields, saw 41

In [ ]:
final_dataframe.keys()

dict_keys(['df1', 'df2', 'df3', 'df4', 'df5'])

In [ ]:
final_dataframe['df1'].head()

,CaseID,PartyID,FileDate,Court,CaseNbr,CaseDisposition,DispositionDate
0,10226263,16090772,01/02/2007,LaPorte Circuit Court,46C01-0701-FD-000018,Guilty Plea or Admission,02-22-2008
1,10226252,16084511,01/02/2007,LaPorte Circuit Court,46C01-0701-FD-000005,Guilty Plea or Admission,09-19-2012
2,10226250,16090225,01/02/2007,LaPorte Circuit Court,46C01-0701-FD-000001,Guilty Plea or Admission,02-06-2008
3,10226251,15681809,01/02/2007,LaPorte Circuit Court,46C01-0701-FD-000004,Guilty Plea or Admission,03-17-2007
4,10226253,27827311,01/02/2007,LaPorte Circuit Court,46C01-0701-FD-000006,Guilty Plea or Admission,03-07-2007


In [ ]:
df_cases = final_dataframe['df1']

In [ ]:
len(df_cases)

3599002

In [ ]:
len(df_cases.CaseID.unique())

3597463

In [ ]:
final_dataframe['df2'].head()

,CaseID,ChargeID,CountNumber,OffenseDate,ChargedStatute,ChargedOffense,ChargedDegree,ConvictedStatute,ConvictedOffense,ConvictedDegree,ConvictedDisposition,ConvictedDate,DisposedStatute,DisposedOffense,DisposedDegree,Disposition,DispositionDate
0,244738,75306,1,12-31-2006,35-43-2-2,TRESPASS,Misdemeanor Class A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,244739,75307,1,12-31-2006,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007
2,244739,75308,2,12-31-2006,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007
3,244740,75309,1,12-28-2006,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007
4,244741,75310,1,01-01-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_charges = final_dataframe['df2']

In [ ]:
len(df_charges.columns)

17

In [ ]:
len(df_charges)

7157306

In [ ]:
len(df_charges.CaseID.unique())

3593497

In [ ]:
final_dataframe['df3'].head()

,CaseID,Category,Description,ChargeAmount,PaymentAmount,CreditAmount,FeeBalance
0,244739,County User Fees,Indiana University Bloomington Police,3.0,3.0,0.0,0.0
1,244739,Filing Fees,Court Costs - City and Town - CR,3.6,3.6,0.0,0.0
2,244739,Filing Fees,Court Costs - County - CR,73.4,73.4,0.0,0.0
3,244739,Filing Fees,Court Costs - State - CR,84.0,84.0,0.0,0.0
4,244739,Other User Fees,Court Administration Fee - CR,3.0,3.0,0.0,0.0


In [ ]:
df_fees = final_dataframe['df3']

In [ ]:
len(df_fees)

32014032

In [ ]:
len(df_fees.CaseID.unique())

2491551

In [ ]:
final_dataframe['df4'].head()

,CaseID,OtherNumberType,OtherNumber
0,27605938,Protection Order Registry,613103
1,27605948,Prosecutor Case Management Number,25-DM8415
2,27605954,Prosecutor Case Management Number,88D01-1705-CM-DM22025
3,27605966,Prosecutor Case Management Number,88C01-1705-F6-DM22022
4,27605973,Prosecutor Case Management Number,49-DM1269387


In [ ]:
df_other = final_dataframe['df4']

In [ ]:
len(df_other)

4289523

In [ ]:
final_dataframe['df5'].head()

,PartyID,NameLast,NameFirst,NameMid,Suffix,DOB,Address,City,State,Zip,Race,Sex
0,217301,Hensel,Richard,NaN,NaN,10/1967,900 E. Main Street,Logansport,IN,46947,White,F
1,217301,Willis,Trinidad,NaN,NaN,10/1967,900 E. Main Street,Logansport,IN,46947,White,F
2,241354,Harding,Travis,Lee,NaN,3/1980,5350 North Lakeview Road,Bloomington,IN,47404,White,M
3,241426,Swartz,Claude,David,NaN,8/1976,17 Eastside Lane,Bedford,IN,47421,White,M
4,241532,Rhodes,George,Justin,NaN,1/1981,"5451 Snapdragon Lane, Apt. 6",Indianapolis,IN,46203,White,M


In [ ]:
df_parties = final_dataframe['df5']

In [ ]:
len(df_parties)

2362973

Creat a total fee per case dataframe

In [ ]:
df_fee_total = df_fees.groupby('CaseID')['FeeBalance'].sum().reset_index()

In [ ]:
len(df_fee_total)

2491551

In [ ]:
df_fee_total.head()

,CaseID,FeeBalance
0,244739,0.0
1,244740,0.0
2,244741,0.0
3,244743,259.0
4,244744,-5.0


#Combining the datasets to include all the relevant information

In [ ]:
#Create dataset of case numbers that includes PartyID
df_case_party = df_cases[['CaseID', 'PartyID']]

In [ ]:
df_case_party['PartyID'] = df_case_party['PartyID'].astype('Int64')
df_charges['ChargeID'] = df_charges['ChargeID'].astype('Int64')
df_charges['CountNumber'] = df_charges['CountNumber'].astype('Int64')

<ipython-input-30-00a170699cb1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case_party['PartyID'] = df_case_party['PartyID'].astype('Int64')


In [ ]:
#Merge party with charge dataset
df_charge_party = pd.merge(df_case_party, df_charges, on="CaseID", how='outer')

In [ ]:
len(df_charge_party)

7166261

In [ ]:
df_charge_party.head()

,CaseID,PartyID,ChargeID,CountNumber,OffenseDate,ChargedStatute,ChargedOffense,ChargedDegree,ConvictedStatute,ConvictedOffense,ConvictedDegree,ConvictedDisposition,ConvictedDate,DisposedStatute,DisposedOffense,DisposedDegree,Disposition,DispositionDate
0,244738,1124445,75306,1,12-31-2006,35-43-2-2,TRESPASS,Misdemeanor Class A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,244739,862741,75307,1,12-31-2006,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007
2,244739,862741,75308,2,12-31-2006,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007
3,244740,862743,75309,1,12-28-2006,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007
4,244741,862744,75310,1,01-01-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Merge with fees dataset
#Merge party with charge dataset
df = pd.merge(df_charge_party, df_fee_total, on="CaseID", how='outer')

In [ ]:
len(df)

7166268

In [ ]:
df.head()

,CaseID,PartyID,ChargeID,CountNumber,OffenseDate,ChargedStatute,ChargedOffense,ChargedDegree,ConvictedStatute,ConvictedOffense,ConvictedDegree,ConvictedDisposition,ConvictedDate,DisposedStatute,DisposedOffense,DisposedDegree,Disposition,DispositionDate,FeeBalance
0,244738,1124445,75306,1,12-31-2006,35-43-2-2,TRESPASS,Misdemeanor Class A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,244739,862741,75307,1,12-31-2006,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007,7.1-5-1-3,PUBLIC INTOXICATION,Misdemeanor Class B,Finding of Guilty,03-14-2007,0.0
2,244739,862741,75308,2,12-31-2006,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,Finding of Guilty,03-14-2007,0.0
3,244740,862743,75309,1,12-28-2006,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007,9-30-5-2,"OPER WHILE INTOX, ENDANGERING PERSONS",Misdemeanor Class A,Finding of Guilty,09-12-2007,0.0
4,244741,862744,75310,1,01-01-2007,7.1-5-7-7,ILLEGAL CONSUMPTION OF AN ALCOHOLIC BEVE,Misdemeanor Class C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
#convert to datetime
df['DispositionDate']= pd.to_datetime(df['DispositionDate'], format='%m-%d-%Y', errors='coerce')


#Save finished dataframe

In [ ]:
df.to_pickle("/content/drive/MyDrive/Expungement/dataframe.pkl")